In [1]:
import sys
sys.path.append('..')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel


2.8.0


In [2]:
dataset = load_fb15k_237()

Jump to 
- [Partitioned](#Training/eval-with-partition)
- [Discovery](#Discovery)

# Train/eval without partition

### Train a random model

In [3]:
optim = tf.optimizers.Adam(learning_rate=0.001)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'

from ampligraph.latent_features.loss_functions import SelfAdversarialLoss, NLLMulticlass
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5, 'reduction': 'sum'})
loss = NLLMulticlass({'reduction': 'mean'})
model = ScoringBasedEmbeddingModel(eta=2, 
                                     k=50,
                                     scoring_type='Random')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

dataset = load_fb15k_237()

start = time.time()
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=25,
             validation_batch_size=100,
             validation_data = dataset['valid'],
         callbacks=[checkpoint])

end = time.time()
print('Time taken:', end-start)

2022-05-06 10:20:58.332400: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 10:20:58.956246: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38238 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:39:00.0, compute capability: 8.0


Epoch 1/10
29/29 [==============================] - 2s 62ms/step - loss: 6982.9214

Epoch 1: loss improved from inf to 6982.92139, saving model to ./chkpt1
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


2022-05-06 10:21:11.531385: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/10
29/29 [==============================] - 1s 33ms/step - loss: 6986.3037

Epoch 2: loss did not improve from 6982.92139
Epoch 3/10
29/29 [==============================] - 1s 33ms/step - loss: 6987.4985

Epoch 3: loss did not improve from 6982.92139
Epoch 4/10
29/29 [==============================] - 1s 33ms/step - loss: 6987.1611

Epoch 4: loss did not improve from 6982.92139
Epoch 5/10
29/29 [==============================] - 1s 33ms/step - loss: 6987.0215

Epoch 5: loss did not improve from 6982.92139
Epoch 6/10
29/29 [==============================] - 1s 33ms/step - loss: 6986.9375

Epoch 6: loss did not improve from 6982.92139
Epoch 7/10
29/29 [==============================] - 1s 33ms/step - loss: 6987.0557

Epoch 7: loss did not improve from 6982.92139
Epoch 8/10
29/29 [==============================] - 1s 33ms/step - loss: 6987.1411

Epoch 8: loss did not improve from 6982.92139
Epoch 9/10
29/29 [==============================] - 1s 32ms/step - loss: 6986.9224

Epoch 

### Evaluate the random model

In [4]:
# Full evaluation (default protocol) using filters
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
mrr_score(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
206/206 [==============================] - 85s 413ms/step


0.0005052450328641726

### Train a model

In [5]:
optim = tf.optimizers.Adam(learning_rate=0.001)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./distmult_logs')

from ampligraph.latent_features.loss_functions import SelfAdversarialLoss, NLLMulticlass
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5, 'reduction': 'mean'})
loss = NLLMulticlass({'reduction': 'mean'})
model = ScoringBasedEmbeddingModel(eta=30, 
                                     k=350,
                                     scoring_type='DistMult')



model.compile(optimizer=optim, loss=loss, entity_relation_regularizer=tf.keras.regularizers.L2(0.0001))

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

dataset = load_fb15k_237()

start = time.time()
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=50,
             validation_batch_size=100,
             validation_data = dataset['valid'],
         callbacks=[checkpoint, tensorboard_callback])

end = time.time()
print('Time taken:', end-start)

# the training can be visualised using the following command:
# tensorboard --logdir='./distmult_logs' --port=8891 

Epoch 1/10
29/29 [==============================] - 2s 52ms/step - loss: 6736.0894

Epoch 1: loss improved from inf to 6736.08936, saving model to ./chkpt1
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.
Epoch 2/10
29/29 [==============================] - 1s 34ms/step - loss: 6730.6094

Epoch 2: loss improved from 6736.08936 to 6730.60938, saving model to ./chkpt1
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.
Epoch 3/10
29/29 [==============================] - 1s 34ms/step - loss: 6694.9629

Epoch 3: loss improved from 6730.60938 to 6694.96289, saving model to ./chkpt1
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.
Epoch 4/10
29/29 [==============================] - 1s 35ms/step - loss: 6556.94

In [6]:
# Full evaluation (default protocol) using filters
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})

mrr_score(ranks), hits_at_n_score(ranks, 1)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
206/206 [==============================] - 85s 415ms/step


(0.2602270358415026, 0.18793424014091398)

### Visualize the embeddings

In [7]:
from ampligraph.utils import create_tensorboard_visualizations

In [8]:
create_tensorboard_visualizations(model, 
                                  entities_subset=['/m/027rn', '/m/06cx9', '/m/017dcd', '/m/06v8s0', '/m/07s9rl0'], 
                                  labels=['ent1', 'ent2', 'ent3', 'ent4', 'ent5'],
                                  loc = './small_embeddings_vis')

In [9]:
create_tensorboard_visualizations(model, 
                                  entities_subset='all',
                                  loc = './full_embeddings_vis')

In [10]:
# the embeddings can be visualised using the following command:
# tensorboard --logdir='./full_embeddings_vis' --port=8891 
# open the browser and go to the following URL: http://127.0.0.1:8891/#projector

### Evaluate

In [11]:
# evaluate using filters
ranks = model.evaluate(np.array([['/m/01cr28', '/location/country/form_of_government', '/m/02lkcc'],
                     ['/m/07tw_b', '/location/country/form_of_government', '/m/02lkcc'],
                     ['/m/073tm9', '/location/country/form_of_government', '/m/02lkcc']]), 
                       batch_size=3,
                       use_filter={'train': dataset['train'],
                                  'test': dataset['test']}, 
                       corrupt_side='s,o', 
                       verbose=True)

mrr_score(ranks), hits_at_n_score(ranks, 10)

2/2 [==============================] - 1s 326ms/step


(0.00023974997689174383, 0.0)

In [12]:
# evaluate using filters (filters are file names instead of numpy arrays)
# corruptions generated using entities subset
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=500,
                       corrupt_side='s,o',
                       entities_subset=['/m/08966', '/m/05lf_', '/m/0f8l9c', '/m/04ghz4m'],
                      
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
mrr_score(ranks), hits_at_n_score(ranks, 1) # will give very high mrr


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
42/42 [==============================] - 80s 2s/step


(0.9626798121146883, 0.9292739015559253)

In [13]:
# Full evaluation (default protocol) using filters
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o')

mrr_score(ranks), hits_at_n_score(ranks, 1)


28 triples containing invalid keys skipped!
      8/Unknown - 0s 51ms/step

2022-05-06 09:44:30.420311: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: RaggedFromRowLengths/assert_equal_1/Assert/AssertGuard/branch_executed/_18


206/206 [==============================] - 4s 18ms/step


(0.10250035630223059, 0.0)

In [14]:
# Full evaluation (default protocol) using filters
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})

mrr_score(ranks), hits_at_n_score(ranks, 1)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
206/206 [==============================] - 85s 413ms/step


(0.2602270358415026, 0.18793424014091398)

In [15]:
# same as above but just for sanity checking if entities_subset works or not
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                       entities_subset=model.data_indexer.backend.get_all_entities(),
                      
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mrr_score(ranks), hits_at_n_score(ranks, 1)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
206/206 [==============================] - 85s 412ms/step
Time taken: 95.00184178352356


(0.2602270358415026, 0.18793424014091398)

In [16]:
np.random.seed(0)
pred = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(np.sort(pred))


28 triples containing invalid keys skipped!
(20438,)
[-2.897735  -1.8650666 -1.8186185 ...  7.398894   7.3988943  7.887154 ]


### Model calibration

In [17]:
# calibrate on the test set
model.calibrate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                batch_size=10000, positive_base_rate=0.9, epochs=100)


28 triples containing invalid keys skipped!


In [18]:
# check if the sorted probability indices match the sorted regular scores 
# It should be same as calibration doesnt change ranking, it just calibrates the range of scores
out = model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
print(np.sort(out))
print(np.argsort(out))
pred_out = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
print(np.sort(pred_out))
print(np.argsort(pred_out))



28 triples containing invalid keys skipped!
[0.3826677  0.44528607 0.44815987 ... 0.89083356 0.89083356 0.9021644 ]
[ 3834 18634  4066 ...  2021  9247 14612]

28 triples containing invalid keys skipped!
[-2.897735  -1.8650666 -1.8186185 ...  7.398894   7.3988943  7.887154 ]
[ 3834 18634  4066 ...  9247  2021 14612]


In [19]:
# calibration should not affect the regular evaluation
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
410/410 [==============================] - 175s 427ms/step
Time taken: 185.4112503528595


(335.33369214208824,
 0.26022703584150253,
 0.18793424014091398,
 0.4024611018690674,
 40876)

In [20]:
model.save_weights('./calibrated_model')

## Load Checkpoint and evaluate

In [21]:
# Loaded the model should return exact same results as earlier
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=2, 
                                     k=350, 
                                     scoring_type='DistMult')
loaded_model.load_weights('./calibrated_model')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
206/206 [==============================] - 85s 412ms/step
Time taken: 94.97969150543213


(335.33369214208824,
 0.2602270358415026,
 0.18793424014091398,
 0.4024611018690674,
 20438)

In [22]:
pred = loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(pred)


28 triples containing invalid keys skipped!
(20438,)
[3.7724204  2.6032825  5.378133   ... 1.8153479  0.29437166 4.713559  ]


In [23]:
# sorted calibration scores order and regular predict scores order must match
out = loaded_model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                 batch_size=10000)
np.argsort(out), \
np.argsort(loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')), \
np.sort(out)



28 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!


(array([ 3834, 18634,  4066, ...,  2021,  9247, 14612]),
 array([ 3834, 18634,  4066, ...,  2021,  9247, 14612]),
 array([0.3826677 , 0.44528604, 0.44815987, ..., 0.89083356, 0.89083356,
        0.9021644 ], dtype=float32))

# Training/eval with partition 

## Training with RandomEdges partitioner

In [4]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                     use_filter=False,
                                    use_indexer=True)


In [5]:
# Choose the partitioner 
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)


_split: memory before: 0.0Bytes, after: 12.81MB, consumed: 12.81MB; exec time: 20.264s


In [6]:

optim = tf.optimizers.Adam(learning_rate=0.001)

partitioned_model = ScoringBasedEmbeddingModel(eta=2, 
                                     k=50, 
                                     scoring_type='DistMult')
partitioned_model.compile(optimizer=optim, loss='multiclass_nll')


2022-05-04 14:48:59.955534: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 14:49:00.580059: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38238 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:39:00.0, compute capability: 8.0


In [7]:

start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,             
                     epochs=10)
print((time.time()-start))


Epoch 1/10
274/274 [==============================] - 26s 96ms/step - loss: 1095.1127
Epoch 2/10
274/274 [==============================] - 26s 93ms/step - loss: 1090.9618
Epoch 3/10
274/274 [==============================] - 26s 96ms/step - loss: 1080.2867
Epoch 4/10
274/274 [==============================] - 26s 95ms/step - loss: 1052.5120
Epoch 5/10
274/274 [==============================] - 26s 93ms/step - loss: 1004.9535
Epoch 6/10
274/274 [==============================] - 25s 93ms/step - loss: 945.0866
Epoch 7/10
274/274 [==============================] - 26s 95ms/step - loss: 880.6619
Epoch 8/10
274/274 [==============================] - 27s 97ms/step - loss: 817.8425
Epoch 9/10
274/274 [==============================] - 26s 95ms/step - loss: 759.7907
Epoch 10/10
274/274 [==============================] - 26s 94ms/step - loss: 707.4430
281.769056558609


In [8]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=partitioned_model.data_handler.get_mapper())



28 triples containing invalid keys skipped!


In [9]:

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


53/53 [==============================] - 227s 4s/step
Time taken: 227.30635476112366


(1171.3525540659555, 0.07391564339336283, 0.0, 0.20427634797925434, 20438)

## Save and Load

In [10]:
partitioned_model.save_weights('./best_model')


In [11]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=2, 
                                     k=50, 
                                     scoring_type='DistMult')

loaded_part_model.load_weights('./best_model')


In [12]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=loaded_part_model.data_indexer)



28 triples containing invalid keys skipped!


In [13]:


start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


53/53 [==============================] - 226s 4s/step
Time taken: 225.89381051063538


(1171.3525540659555, 0.07391564339336283, 0.0, 0.20427634797925434, 20438)

# Training/eval with partition (default Partitioning Approach)

In [14]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY


In [15]:
optim = tf.optimizers.Adam(learning_rate=0.001)

partitioned_model = ScoringBasedEmbeddingModel(eta=2, 
                                     k=50, 
                                     scoring_type='DistMult')
partitioned_model.compile(optimizer=optim, loss='multiclass_nll')


In [16]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs_split')
start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=10000, use_partitioning=True,
                     epochs=10, callbacks=[tensorboard_callback])
print((time.time()-start))


_split: memory before: 31.779MB, after: 44.602MB, consumed: 12.823MB; exec time: 150.55s
Epoch 1/10
     10/Unknown - 1s 112ms/step - loss: 9905.5361 2 2
     12/Unknown - 3s 261ms/step - loss: 10093.00002 2
     13/Unknown - 4s 337ms/step - loss: 10167.41502 2
     23/Unknown - 6s 258ms/step - loss: 9672.1377 2 2
     28/Unknown - 7s 261ms/step - loss: 9846.09282 2
     31/Unknown - 9s 279ms/step - loss: 9960.09382 2
32/32 [==============================] - 9s 292ms/step - loss: 9643.4775
Epoch 2/10
32/32 [==============================] - 8s 257ms/step - loss: 9643.3975
Epoch 3/10
32/32 [==============================] - 8s 263ms/step - loss: 9643.2178
Epoch 4/10
32/32 [==============================] - 8s 265ms/step - loss: 9642.8262
Epoch 5/10
32/32 [==============================] - 9s 268ms/step - loss: 9641.9248
Epoch 6/10
32/32 [==============================] - 8s 262ms/step - loss: 9639.8564
Epoch 7/10
32/32 [==============================] - 8s 262ms/step - loss: 9635.4365
E

In [17]:
len(set(dataset['train'][:, 0]).union(set(dataset['train'][:, 2])))


14505

In [18]:


start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!
53/53 [==============================] - 230s 4s/step
Time taken: 230.9751660823822


(1335.5136999706428, 0.06652368353952463, 0.0, 0.1842646051472747, 20438)

## Filtered evaluation

In [19]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=400,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)



28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!
53/53 [==============================] - 431s 8s/step
Time taken: 451.3540623188019


(1194.4257021234955,
 0.15864779365563283,
 0.10859673157843233,
 0.2576817692533516,
 20438)

## Random model with partitioning

In [ ]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY

optim = tf.optimizers.Adam(learning_rate=0.001)

partitioned_model = ScoringBasedEmbeddingModel(eta=2, 
                                     k=50, 
                                     scoring_type='Random')
partitioned_model.compile(optimizer=optim, loss='multiclass_nll')


start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=10000, use_partitioning=True,
                     epochs=10, callbacks=[])
print((time.time() - start))


In [ ]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


# Discovery

### Discover Facts

In [4]:
from ampligraph.discovery import discover_facts

model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=50,
                                     scoring_type='ComplEx')



model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), 
              loss='multiclass_nll')

start = time.time()
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=50,
             validation_batch_size=100,
             validation_data = dataset['valid'])

end = time.time()
print('Time taken:', end-start)

discover_facts(dataset['train'][:100], 
               model, 
               top_n=100, 
               strategy='random_uniform', 
               max_candidates=100, 
               target_rel='/location/country/form_of_government', 
               seed=0)


Epoch 1/10
29/29 [==============================] - 3s 88ms/step - loss: 17412.8789
Epoch 2/10
29/29 [==============================] - 1s 39ms/step - loss: 17409.4043
Epoch 3/10
29/29 [==============================] - 1s 38ms/step - loss: 17388.6035
Epoch 4/10
29/29 [==============================] - 1s 41ms/step - loss: 17307.3789
Epoch 5/10
29/29 [==============================] - 1s 39ms/step - loss: 17095.1973
Epoch 6/10
29/29 [==============================] - 1s 39ms/step - loss: 16681.0527
Epoch 7/10
29/29 [==============================] - 1s 41ms/step - loss: 16038.2012
Epoch 8/10
29/29 [==============================] - 1s 37ms/step - loss: 15221.4160
Epoch 9/10
29/29 [==============================] - 1s 40ms/step - loss: 14335.6729
Epoch 10/10
29/29 [==============================] - 1s 41ms/step - loss: 13471.0947
Time taken: 24.852190494537354


(array([['/m/0fvf9q', '/location/country/form_of_government', '/m/0k049']],
       dtype=object),
 array([94.]))

### Find Clusters

In [43]:
import requests
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns


from ampligraph.datasets import load_from_csv
from ampligraph.discovery import find_clusters

# International football matches triples
# See tutorial here to understand how the triples are created from a tabular dataset:
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/football.csv'
open('football.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'football.csv', sep=',')[:, 1:]

model = ScoringBasedEmbeddingModel(eta=5, 
                                 k=300,
                                 scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')

model.fit(X,
          batch_size=10000,
          epochs=10)

df = pd.DataFrame(X, columns=["s", "p", "o"])
teams = np.unique(np.concatenate((df.s[df.s.str.startswith("Team")],
                               df.o[df.o.str.startswith("Team")])))
team_embeddings = model.get_embeddings(teams, embedding_type='e')

embeddings_2d = PCA(n_components=2).fit_transform(np.array([i for i in team_embeddings]))

# Find clusters of embeddings using KMeans

kmeans = KMeans(n_clusters=6, n_init=100, max_iter=500)
clusters = find_clusters(teams, model, kmeans, mode='e')
print(np.unique(clusters, return_counts=True))


Epoch 1/10
46/46 [==============================] - 6s 128ms/step - loss: 17856.8340
Epoch 2/10
46/46 [==============================] - 5s 109ms/step - loss: 17844.8906
Epoch 3/10
46/46 [==============================] - 5s 109ms/step - loss: 17735.7676
Epoch 4/10
46/46 [==============================] - 5s 109ms/step - loss: 17341.3867
Epoch 5/10
46/46 [==============================] - 5s 110ms/step - loss: 16681.2773
Epoch 6/10
46/46 [==============================] - 5s 107ms/step - loss: 15790.5664
Epoch 7/10
46/46 [==============================] - 5s 110ms/step - loss: 14691.8779
Epoch 8/10
46/46 [==============================] - 5s 108ms/step - loss: 13537.4414
Epoch 9/10
46/46 [==============================] - 5s 110ms/step - loss: 12469.3301
Epoch 10/10
46/46 [==============================] - 5s 111ms/step - loss: 11533.4336
(array([0, 1, 2, 3, 4, 5], dtype=int32), array([ 49,   1,  56,  54,  43, 110]))


### Find Duplicates

In [44]:
import pandas as pd
import numpy as np
import re

# The IMDB dataset used here is part of the Movies5 dataset found on:
# The Magellan Data Repository (https://sites.google.com/site/anhaidgroup/projects/data)
import requests
url = 'http://pages.cs.wisc.edu/~anhai/data/784_data/movies5.tar.gz'
open('movies5.tar.gz', 'wb').write(requests.get(url).content)
import tarfile
tar = tarfile.open('movies5.tar.gz', "r:gz")
tar.extractall()
tar.close()

# Reading tabular dataset of IMDB movies and filling the missing values
imdb = pd.read_csv("movies5/csv_files/imdb.csv")
imdb["directors"] = imdb["directors"].fillna("UnknownDirector")
imdb["actors"] = imdb["actors"].fillna("UnknownActor")
imdb["genre"] = imdb["genre"].fillna("UnknownGenre")
imdb["duration"] = imdb["duration"].fillna("0")

# Creating knowledge graph triples from tabular dataset
imdb_triples = []

for _, row in imdb.iterrows():
    movie_id = "ID" + str(row["id"])
    directors = row["directors"].split(",")
    actors = row["actors"].split(",")
    genres = row["genre"].split(",")
    duration = "Duration" + str(int(re.sub("\D", "", row["duration"])) // 30)

    directors_triples = [(movie_id, "hasDirector", d) for d in directors]
    actors_triples = [(movie_id, "hasActor", a) for a in actors]
    genres_triples = [(movie_id, "hasGenre", g) for g in genres]
    duration_triple = (movie_id, "hasDuration", duration)

    imdb_triples.extend(directors_triples)
    imdb_triples.extend(actors_triples)
    imdb_triples.extend(genres_triples)
    imdb_triples.append(duration_triple)

# Training knowledge graph embedding with ComplEx model
model = ScoringBasedEmbeddingModel(eta=5, 
                             k=300,
                             scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')
imdb_triples = np.array(imdb_triples)
model.fit(imdb_triples,
      batch_size=10000,
      epochs=20)

# Finding duplicates movies (entities)
from ampligraph.discovery import find_duplicates

entities = np.unique(imdb_triples[:, 0])
dups, _ = find_duplicates(entities, model, mode='e', tolerance=0.45)
id_list = []
for data in dups:
    for i in data:
        id_list.append(int(i[2:]))
print(imdb.iloc[id_list[:6]][['movie_name', 'year']])


Epoch 1/20
7/7 [==============================] - 1s 208ms/step - loss: 15612.8779
Epoch 2/20
7/7 [==============================] - 1s 90ms/step - loss: 15610.4873
Epoch 3/20
7/7 [==============================] - 1s 87ms/step - loss: 15607.6924
Epoch 4/20
7/7 [==============================] - 1s 88ms/step - loss: 15603.9346
Epoch 5/20
7/7 [==============================] - 1s 90ms/step - loss: 15598.6699
Epoch 6/20
7/7 [==============================] - 1s 88ms/step - loss: 15591.2236
Epoch 7/20
7/7 [==============================] - 1s 88ms/step - loss: 15580.7832
Epoch 8/20
7/7 [==============================] - 1s 92ms/step - loss: 15566.3682
Epoch 9/20
7/7 [==============================] - 1s 87ms/step - loss: 15546.8447
Epoch 10/20
7/7 [==============================] - 1s 86ms/step - loss: 15521.0059
Epoch 11/20
7/7 [==============================] - 1s 88ms/step - loss: 15487.4824
Epoch 12/20
7/7 [==============================] - 1s 86ms/step - loss: 15444.7588
Epoch 13/20


### Query TopN

In [45]:
import requests
from ampligraph.datasets import load_from_csv
from ampligraph.discovery import discover_facts
from ampligraph.discovery import query_topn

# Game of Thrones relations dataset
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'GoT.csv', sep=',')

model = ScoringBasedEmbeddingModel(eta=5, 
                             k=150,
                             scoring_type='DistMult')



model.compile(optimizer='Adam', loss='pairwise')
model.fit(X,
      batch_size=100,
      epochs=20)
query_topn(model, top_n=5,
        head='Eddard Stark', relation='ALLIED_WITH', tail=None,
        ents_to_consider=None, rels_to_consider=None)


Epoch 1/20
33/33 [==============================] - 1s 25ms/step - loss: 496.0767
Epoch 2/20
33/33 [==============================] - 0s 6ms/step - loss: 495.4716
Epoch 3/20
33/33 [==============================] - 0s 6ms/step - loss: 494.6113
Epoch 4/20
33/33 [==============================] - 0s 6ms/step - loss: 493.0753
Epoch 5/20
33/33 [==============================] - 0s 6ms/step - loss: 490.1469
Epoch 6/20
33/33 [==============================] - 0s 6ms/step - loss: 484.6407
Epoch 7/20
33/33 [==============================] - 0s 6ms/step - loss: 474.8754
Epoch 8/20
33/33 [==============================] - 0s 6ms/step - loss: 458.7555
Epoch 9/20
33/33 [==============================] - 0s 6ms/step - loss: 434.9001
Epoch 10/20
33/33 [==============================] - 0s 7ms/step - loss: 407.1082
Epoch 11/20
33/33 [==============================] - 0s 7ms/step - loss: 378.7333
Epoch 12/20
33/33 [==============================] - 0s 7ms/step - loss: 351.8762
Epoch 13/20
33/33 [=====

(array([['Eddard Stark', 'ALLIED_WITH', 'House Stark of Winterfell'],
        ['Eddard Stark', 'ALLIED_WITH', 'The Vale'],
        ['Eddard Stark', 'ALLIED_WITH', 'House Goodbrother of Hammerhorn'],
        ['Eddard Stark', 'ALLIED_WITH', 'House Locke of Oldcastle'],
        ['Eddard Stark', 'ALLIED_WITH', 'House Greyjoy of Pyke']],
       dtype='<U44'),
 array([2.2265291 , 0.5242001 , 0.5207645 , 0.48392776, 0.46780267],
       dtype=float32))